In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)|

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emoji-data-descriptions-codepoints/emoji_testpage_screnshot.png
/kaggle/input/emoji-data-descriptions-codepoints/emoji_df.csv
/kaggle/input/emoji-data-descriptions-codepoints/emoji-test.txt


In [6]:
import pandas as pd
from datasets import Dataset
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, GPT2LMHeadModel, GPT2Tokenizer
from huggingface_hub import notebook_login

# Install huggingface_hub if not installed
!pip install huggingface_hub

# Log in to the Hugging Face Hub
notebook_login()

def fine_tune_gpt2(dataset_path, model_name='gpt2', output_dir='./results', hub_model_name='my-awesome-model'):
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)

    # Set the padding token
    tokenizer.pad_token = tokenizer.eos_token

    def tokenize_function(examples):
        return tokenizer(examples["Text"], padding="max_length", truncation=True)

    # Load the dataset and convert it to a format suitable for Dataset
    df = pd.read_csv(dataset_path)
    df['Text'] = df['name'] + ' ' + df['emoji']
    dataset = Dataset.from_pandas(df)

    tokenized_dataset = dataset.map(tokenize_function, batched=True)

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=2,  # Experiment with the number of epochs
        per_device_train_batch_size=1,
        save_steps=2000,  # Save model after every 2000 training steps
        save_total_limit=2,
        learning_rate=5e-5,  # Experiment with the learning rate
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=tokenized_dataset,
    )

    # Train the model
    trainer.train()

    # Save the model and tokenizer locally
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    # Push the model and tokenizer to the Hugging Face Hub
    model.push_to_hub(hub_model_name)
    tokenizer.push_to_hub(hub_model_name)

    print(f"Model and tokenizer pushed to the Hub with the name: {hub_model_name}")

fine_tune_gpt2('/kaggle/input/emoji-data-descriptions-codepoints/emoji_df.csv')

  0%|          | 0/5 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,2.742500
1000,2.142500
1500,1.956900
2000,1.838600
2500,1.685500
3000,1.291000
3500,1.360000
4000,1.356100
4500,1.352800


Checkpoint destination directory ./results/checkpoint-2000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Checkpoint destination directory ./results/checkpoint-4000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Model and tokenizer pushed to the Hub with the name: my-awesome-model


In [3]:
# Install huggingface_hub if not installed
!pip install huggingface_hub

# Log in to the Hugging Face Hub
notebook_login()

In [8]:
from transformers import pipeline
def generate_emoji(text, model_path='./results'):
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    model = GPT2LMHeadModel.from_pretrained(model_path)
    generator = pipeline('text-generation', model=model, tokenizer=tokenizer)
    result = generator(text, max_length=50, do_sample=True, temperature=0.7)
    return result[0]['generated_text']

# Test the model
print(generate_emoji('What is up?'))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is up? 🗘️‍🔧️‍🧑‍🧑‍🧑‍🧑‍🧑‍🧑‍🧑�
